In [1]:
%pip install TensorFlow
%pip install Keras
%pip install pillow
%pip install NumPy
%pip install tqdm
%pip install jupyterlab
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras.applications.xception import Xception #to get pre-trained model Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer #for text tokenization
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import add
from keras.models import Model, load_model
from keras.layers import Input, Dense#Keras to build our CNN and LSTM
from keras.layers import LSTM, Embedding, Dropout
from tqdm.notebook import tqdm
tqdm.pandas()
%pip install ipywidgets --upgrade
%pip install tqdm --upgrade


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
%reload_ext autoreload

In [5]:
# Load the document file into memory
def load_fp(filename):
  # Open file to read
   file = open(filename, 'r')
   text = file.read()
   file.close()
   return text


In [6]:
#get all images with their captions
def img_capt(filename):
    file = load_fp(filename)
    captions = file.split('\n')
    descriptions ={}
    for caption in captions[:-1]:
        
        img, caption = caption.split('\t',1)
        if img[:-2] not in descriptions:
           descriptions[img[:-2]] = [ caption ]
        else:
           descriptions[img[:-2]].append(caption)
    return descriptions


In [7]:
#Data cleaning function will convert all upper case alphabets to lowercase, removing punctuations and words containing numbers
def txt_clean(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):
           img_caption.replace("-"," ")
           descp = img_caption.split()
          #uppercase to lowercase
           descp = [wrd.lower() for wrd in descp]
          #remove punctuation from each token
           descp = [wrd.translate(table) for wrd in descp]
          #remove hanging 's and a
           descp = [wrd for wrd in descp if(len(wrd)>1)]
          #remove words containing numbers with them
           descp = [wrd for wrd in descp if(wrd.isalpha())]
          #converting back to string
           img_caption = ' '.join(descp)
           captions[img][i]= img_caption
    return captions


In [8]:
def txt_vocab(descriptions):
  # To build vocab of all unique words
    vocab = set()
    for key in descriptions.keys():
       [vocab.update(d.split()) for d in descriptions[key]]
    return vocab
#To save all descriptions in one file
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
           lines.append(key + 't' + desc )
    data = "n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()


In [10]:
# Set these path according to project folder in you system, like i create a folder with my name shikha inside D-drive
dataset_text = r"C:/work/Image-to-Caption/Flickr8k_text"
dataset_images =r"C:/work/Image-to-Caption/Flicker8k_Dataset"
#to prepare our text data
filename = dataset_text + "/Flickr8k.token.txt"
print(filename +" This is filr")
#loading the file that contains all data
#map them into descriptions dictionary 
descriptions = img_capt(filename)
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = txt_clean(descriptions)
#to build vocabulary
vocabulary = txt_vocab(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))
#saving all descriptions in one file
save_descriptions(clean_descriptions, "descriptions.txt")

C:/work/Image-to-Caption/Flickr8k_text/Flickr8k.token.txt This is filr
Length of descriptions = 8092
Length of vocabulary =  8763


In [26]:
def extract_features(directory):
    print("This is directory"+ directory)
    model = Xception(include_top=True)

    features = {}
    for pic in tqdm(os.listdir(directory)):
        print(f"this is dir {directory}" )
        file = directory + "/" + '1000268201_693b08cb0e.jpg'
        
        image = Image.open(file)
        
        
        print("Sm")
        image = image.resize((299, 299))
        image = np.expand_dims(image, axis=0)

        # image = preprocess_input(image) 
        image = image / 127.5
        image = image - 1.0
        feature = model.predict(image)
        features[pic] = feature
        
    return features

In [27]:
# 2048 feature vector
features = extract_features(dataset_images)
dump(features, open("features.p", "wb"))

This is directoryC:/work/Image-to-Caption/Flicker8k_Dataset


  0%|          | 0/8091 [00:00<?, ?it/s]

this is dir C:/work/Image-to-Caption/Flicker8k_Dataset
Sm


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U13'), dtype('uint8')) -> None

In [15]:
# to directly load the features from the pickle file.
features = load(open("features.p", "rb"))

def load_photos(filename):
    file = load_fp(filename)
    photos = file.split("n")[:-1]
    return photos

FileNotFoundError: [Errno 2] No such file or directory: 'features.p'

In [11]:
def load_clean_descriptions(filename, photos):
    # loading clean_descriptions
    file = load_fp(filename)
    descriptions = {}
    for line in file.split("n"):
        words = line.split()
        if len(words) < 1:
            continue
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = ' ' + " ".join(image_caption) + ' '
            descriptions[image].append(desc)
    return descriptions

def load_features(photos):
    # loading all features
    all_features = load(open("features.p", "rb"))
    # selecting only needed features
    features = {k: all_features[k] for k in photos}
    return features

filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"
# train = loading_data(filename)
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)
train_features = load_features(train_imgs)



83683744/83683744 [==============================] - 6s 0us/step


  0%|          | 0/8091 [00:00<?, ?it/s]

this is dir C:/work/Image-to-Caption/Flicker8k_Dataset
Sm


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U13'), dtype('uint8')) -> None